https://pub.towardsai.net/the-python-record-linkage-toolkit-9b1c59fd156a#:~:text=Record%20linkage%20is%20the%20process,referring%20to%20the%20same%20entity.

In [1]:
#pip install recordlinkage


                                              0.0/926.9 kB ? eta -:--:--
     -                                       41.0/926.9 kB 1.9 MB/s eta 0:00:01
     -                                       41.0/926.9 kB 1.9 MB/s eta 0:00:01
     -----                                  122.9/926.9 kB 1.0 MB/s eta 0:00:01
     -------                                194.6/926.9 kB 1.2 MB/s eta 0:00:01
     ----------                             256.0/926.9 kB 1.2 MB/s eta 0:00:01
     -------------                          317.4/926.9 kB 1.2 MB/s eta 0:00:01
     -------------                          337.9/926.9 kB 1.2 MB/s eta 0:00:01
     ---------------                        368.6/926.9 kB 1.1 MB/s eta 0:00:01
     ---------------                        368.6/926.9 kB 1.1 MB/s eta 0:00:01
     -------------------                    481.3/926.9 kB 1.1 MB/s eta 0:00:01
     ----------------------                 542.7/926.9 kB 1.1 MB/s eta 0:00:01
     ------------------------               59


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\mbieg\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import pandas
import recordlinkage
from recordlinkage.preprocessing import clean

names=['Towards AI','Artificial %%Intelligence','Chetan (Ambi)','$&& Machine %^&* Learning']
s = pandas.Series(names)
print(clean(s))

0                 towards ai
1    artificial intelligence
2                     chetan
3           machine learning
dtype: object


In [7]:
import recordlinkage
from recordlinkage.datasets import load_febrl4

In [8]:

dfA, dfB = load_febrl4()

dfA.shape, dfB.shape

((5000, 10), (5000, 10))

In [9]:
dfA.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168
rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102
rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688


In [10]:
dfB.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-561-dup-0,elton,NaN,3,light setreet,pinehill,windermere,3212,vic,19651013,1551941
rec-2642-dup-0,mitchell,maxon,47,edkins street,lochaoair,north ryde,3355,nsw,19390212,8859999
rec-608-dup-0,NaN,white,72,lambrigg street,kelgoola,broadbeach waters,3159,vic,19620216,9731855
rec-3239-dup-0,elk i,menzies,1,lyster place,NaN,northwood,2585,vic,19980624,4970481
rec-2886-dup-0,NaN,garanggar,NaN,may maxwell crescent,springettst arcade,forest hill,2342,vic,19921016,1366884


In [11]:
indexer = recordlinkage.Index()
indexer.block('given_name')
candidate_links = indexer.index(dfA, dfB)
print("candidate_links:", len(candidate_links))

candidate_links: 77249


In [12]:
compare_cl = recordlinkage.Compare()
compare_cl.exact('given_name', 'given_name', label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1', threshold=0.85, label='address_1')
features = compare_cl.compute(candidate_links, dfA, dfB)

In [13]:
features

given_name  surname  ...  state  address_1
rec_id_1     rec_id_2                             ...                  
rec-1070-org rec-3024-dup-0           1      0.0  ...      1        0.0
             rec-2371-dup-0           1      0.0  ...      0        0.0
             rec-4652-dup-0           1      0.0  ...      0        0.0
             rec-4795-dup-0           1      0.0  ...      1        0.0
             rec-1314-dup-0           1      0.0  ...      1        0.0
...                                 ...      ...  ...    ...        ...
rec-4528-org rec-4528-dup-0           1      1.0  ...      1        1.0
rec-4887-org rec-4887-dup-0           1      1.0  ...      1        1.0
rec-4350-org rec-4350-dup-0           1      1.0  ...      1        1.0
rec-4569-org rec-4569-dup-0           1      1.0  ...      1        0.0
rec-3125-org rec-3125-dup-0           1      1.0  ...      1        1.0

[77249 rows x 6 columns]

In [14]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

6.0     1566
5.0     1332
4.0      343
3.0      146
2.0    16427
1.0    57435
Name: count, dtype: int64

In [15]:
matches=features[features.sum(axis=1)>4]
print(len(matches))

2898


In [16]:
matches.head(10)


,,given_name,surname,date_of_birth,suburb,state,address_1
rec_id_1,rec_id_2,,,,,,
rec-2371-org,rec-2371-dup-0,1,1.0,1,1,1,1.0
rec-4652-org,rec-4652-dup-0,1,1.0,1,0,1,1.0
rec-4795-org,rec-4795-dup-0,1,1.0,1,1,1,1.0
rec-1016-org,rec-1016-dup-0,1,1.0,1,1,0,1.0
rec-2463-org,rec-2463-dup-0,1,1.0,0,1,1,1.0
rec-521-org,rec-521-dup-0,1,1.0,1,1,1,1.0
rec-786-org,rec-786-dup-0,1,1.0,1,1,1,1.0
rec-2389-org,rec-2389-dup-0,1,1.0,1,1,1,1.0
rec-1209-org,rec-1209-dup-0,1,1.0,1,1,1,0.0


In [17]:
dfA.loc['rec-2371-org']

given_name           michaela
surname              dunstone
street_number              37
address_1        deane street
address_2            rosedown
suburb              woodcroft
postcode                 2065
state                     vic
date_of_birth        19121018
soc_sec_id            3166178
Name: rec-2371-org, dtype: object

In [18]:
dfB.loc['rec-2371-dup-0']

given_name           michaela
surname              dunstone
street_number              37
address_1        deane street
address_2            rosedlwn
suburb              woodcroft
postcode                 2065
state                     vic
date_of_birth        19121018
soc_sec_id            3166178
Name: rec-2371-dup-0, dtype: object